# Dota 2 Autoencoder and clustering analyzer

## Paths and Variables

In [ ]:
from dota import Dota2
from clustering import Dota2Clustering
from model import Dota2Autoencoder
import permutations
import torch
import numpy as np
import polars as pl
_patch = [55]
# Patch 7.35, professional matches, 30-120 minutes
dota = Dota2(_patch, ['professional'], (30, 120))
dataset_path = "./tmp/dota_7_33_pro_30_120.csv"
metadata_path = "./tmp/dota_7_33_pro_30_120_metadata.json"
model_path = "./tmp/dota_7_33_pro_30_120_model.h5"
loss_path = "./tmp/dota_7_33_pro_30_120_losses.csv"
plot_path = "./tmp/dota_7_33_pro_30_120_plot.png"
report_path = "./tmp/dota_7_33_pro_30_120_report.txt"

## Load Dataset

In [ ]:
dataset, _ = dota.load_or_prepare_dataset(dataset_path, metadata_path)
train_df, val_df, test_df = dota.prepare_data_splits(dataset, 1.0)

# Autoencoder

### Train/Load Autoencoder

In [ ]:
print("Training models...")
best_permutation, loss_path, model_path, plot_path, report_path = permutations.run_permutations(
    dota, _patch, train_df, val_df, test_df, 100, "./tmp/model_perm", "./tmp/loss_perm", "./tmp/permutation_log")

In [ ]:
print ("Best permutation:", best_permutation)
print("Loading best model...")
model = Dota2Autoencoder.load_model(model_path)


## Clustering

### K-Means

In [ ]:
autoencoder = Dota2Autoencoder.load_model(model_path)
kmeans = Dota2Clustering(dota.autoencoder)
kmeans.cluster(dataset, n_clusters=10)